## CMSC 35300 Final Project: Lasso Models
Shweta Kamath <br>
Nivedita Vatsa <br>
Carolyn Vilter

#### Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import data
df = pd.read_csv("data/all_data_standardized.csv")

In [3]:
# Separate out Xs
X = df.loc[:, ~df.columns.isin(["child_id", "mother_id", "treat_alike_scale", "treat_alike_binary"])]
X = X.to_numpy()

# Separate out two prospective ys
y_scale = df.loc[:, df.columns == "treat_alike_scale"]
y_scale = y_scale.to_numpy()

y_binary = df.loc[:, df.columns == "treat_alike_binary"]
y_binary = y_binary.to_numpy()

### Lasso Regression
Predict repeatedly using cross validation; plot test error.

Source (blog + GitHub):
* https://towardsdatascience.com/regularized-linear-regression-models-dcf5aa662ab9
* https://gist.github.com/wyattowalsh/6a95b1c9ad6118b196336cffd5de4f72

In [4]:
def lasso(X, y, l1, tol=1e-6, path_length=100, return_path=False):
    """The Lasso Regression model with intercept term.
    Intercept term included via design matrix augmentation.
    Pathwise coordinate descent with co-variance updates is applied.
    Path from max value of the L1 tuning parameter to input tuning parameter value.
    Features must be standardized (centered and scaled to unit variance)
    Params:
        X - NumPy matrix, size (N, p), of standardized numerical predictors
        y - NumPy array, length N, of numerical response
        l1 - L1 penalty tuning parameter (positive scalar)
        tol - Coordinate Descent convergence tolerance (exited if change < tol)
        path_length - Number of tuning parameter values to include in path (positive integer)
        return_path - Boolean indicating whether model coefficients along path should be returned
    Returns:
        if return_path == False:
            NumPy array, length p + 1, of fitted model coefficients
        if return_path == True:
            List, length 3, of last fitted model coefficients, tuning parameter path and coefficient values
    """
    X = np.hstack((np.ones((len(X), 1)), X))
    m, n = np.shape(X)
    B_star = np.zeros((n))
    l_max = max(list(abs(np.dot(np.transpose(X[:, 1:]), y)))) / m
    # At or above l_max, all coefficients (except intercept) will be brought to 0
    if l1 >= l_max:
        return np.append(np.mean(y), np.zeros((n - 1)))
    l_path = np.geomspace(l_max, l1, path_length)
    coeffiecients = np.zeros((len(l_path), n))
    for i in range(len(l_path)):
        while True:
            B_s = B_star
            for j in range(n):
                k = np.where(B_s != 0)[0]
                update = (1/m)*((np.dot(X[:,j], y)- \
                                np.dot(np.dot(X[:,j], X[:,k]), B_s[k]))) + \
                                B_s[j]
                B_star[j] = (np.sign(update) * max(abs(update) - l_path[i], 0))
            if np.all(abs(B_s - B_star) < tol):
                coeffiecients[i, :] = B_star
                break
    if return_path:
        return [B_star, l_path, coeffiecients]
    else:
        return B_star

In [ ]:
rv = lasso(X, y_scale, l1=0.1, tol=1e-6, path_length=10, return_path=True)

In [36]:
X_ = np.hstack((np.ones((len(X), 1)), X))

In [37]:
w_vec = rv[0].reshape(len(rv[0]),1)

In [39]:
w_vec

array([[-8.75332096e+149],
       [-1.89954341e+151],
       [ 5.11726785e+155],
       [-1.42027326e+159],
       [ 5.85572677e+164],
       [-7.53499916e+167],
       [-2.31662432e+168],
       [ 1.01670780e+170],
       [-2.57430171e+171],
       [ 9.94662103e+171],
       [-4.36748411e+172],
       [ 1.81239033e+173],
       [-9.45369670e+173],
       [ 5.79481032e+174],
       [-2.81822500e+175],
       [ 1.23979097e+176],
       [-7.40648440e+176],
       [ 5.57228707e+177],
       [-2.99609595e+178],
       [ 1.49837860e+179],
       [-9.78917627e+179],
       [ 7.68455017e+180],
       [-3.98837248e+181],
       [ 2.04284559e+182],
       [-6.09579665e+185],
       [ 8.49306795e+189],
       [-1.75883734e+190],
       [-3.99865247e+195],
       [ 3.32254675e+199],
       [ 1.23859887e+200],
       [ 5.97836911e+199],
       [ 4.12248718e+198],
       [ 3.50576406e+198],
       [ 3.76690870e+199],
       [ 4.18306818e+199],
       [ 1.39670232e+198],
       [ 5.96973049e+198],
 

-----------------------

Source: https://xavierbourretsicotte.github.io/lasso_implementation.html

In [41]:
m, n = X_.shape
num_iters = 2
w_list = []

w = np.ones((n,1))
lamda_list = [0]
#lamda_list = np.logspace(0,4,300)/10 #Range of lambda values

for lamda in lamda_list:
    print("="*10,"lambda:",lamda,"="*10)
    for i in range(num_iters): # iteration
        for j in range(n): # column
            y_scale_pred = X_ @ w

            X_j = X_[:,j].reshape(-1,1)
            
            rho = X_j.T @ (y_scale - y_scale_pred + w[j]*X_j)
            # print(rho)
            
            # update weight vector
            if j == 0: 
                w[j] = rho # intercept term
            else:
#                 if rho < (-lamda/2):
#                     w[j] = rho + (lamda/2)
#                 elif rho > (lamda/2):
#                     w[j] = rho - (lamda/2)
#                 else:
#                     w[j] = 0
                if rho < (-lamda):
                    w[j] = rho + lamda
                elif rho >  lamda:
                    w[j] = rho - lamda
                else:
                    w[j] = 0
        print("j:", j)
        print("w:", w)
        w_list.append(w)

    
#print(w_list)

========== lambda: 0 ==========


ValueError: shapes (5161,1) and (44,1) not aligned: 1 (dim 1) != 44 (dim 0)

In [ ]:
w_lasso = np.stack(w_list).T
X_names = df.columns[(~df.columns.isin(["child_id", "mother_id", "treat_alike_scale", "treat_alike_binary"]))]


#Plot results
n,_ = w_lasso.shape
plt.figure(figsize = (12,8))

for i in range(n):
    plt.plot(lamda, theta_lasso[i], label = X_names[i])

plt.xscale('log')
plt.xlabel('Log($\\lambda$)')
plt.ylabel('Coefficients')
plt.title('Lasso Paths - Numpy implementation')
plt.legend()
plt.axis('tight')

In [35]:
class Regression:
    def __init__(self, learning_rate, iteration, regularization):
        """
        :param learning_rate: A samll value needed for gradient decent, default value id 0.1.
        :param iteration: Number of training iteration, default value is 10,000.
        """
        self.m = None
        self.n = None
        self.w = None
        self.b = None
        self.regularization = regularization # will be the l1/l2 regularization class according to the regression model.
        self.lr = learning_rate
        self.it = iteration

    def cost_function(self, y, y_pred):
        """
        :param y: Original target value.
        :param y_pred: predicted target value.
        """
        # return np.sum(np.square(y_pred - y)) + self.regularization(self.w)
        return (1 / (2*self.m)) * np.sum(np.square(y_pred - y)) + self.regularization(self.w)
    
    def hypothesis(self, weights, bias, X):
        """
        :param weights: parameter value weight.
        :param X: Training samples.
        """
        return np.dot(X, weights) #+ bias

    def train(self, X, y):
        """
        :param X: training data feature values ---> N Dimentional vector.
        :param y: training data target value -----> 1 Dimentional array.
        """
        # Insert constant ones for bias weights.
        X = np.insert(X, 0, 1, axis=1)

        # Target value should be in the shape of (n, 1) not (n, ).
        # So, this will check that and change the shape to (n, 1), if not.
        try:
            y.shape[1]
        except IndexError as e:
            # we need to change it to the 1 D array, not a list.
            print("ERROR: Target array should be a one dimentional array not a list"
                  "----> here the target value not in the shape of (n,1). \nShape ({shape_y_0},1) and {shape_y} not match"
                  .format(shape_y_0 = y.shape[0] , shape_y = y.shape))
            return 
        
        # m is the number of training samples.
        self.m = X.shape[0]
        # n is the number of features.
        self.n = X.shape[1]

        # Set the initial weight.
        self.w = np.zeros((self.n , 1)) + 1

        # bias.
        self.b = 0

        for it in range(1, self.it+1):
            # 1. Find the predicted value through the hypothesis.
            # 2. Find the Cost function value.
            # 3. Find the derivation of weights.
            # 4. Apply Gradient Decent.
            y_pred = self.hypothesis(self.w, self.b, X)
            #print("iteration",it)
            #print("y predict value",y_pred)
            cost = self.cost_function(y, y_pred)
            #print("Cost function",cost)
            # fin the derivative.
            dw = np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.w)
            # dw = (1/self.m) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.w)
            #print("weights derivation",dw)
            #db = -(2 / self.m) * np.sum((y_pred - y))

            # change the weight parameter.
            self.w = self.w - self.lr * dw
            #print("updated weights",self.w)
            #self.b = self.b - self.lr * db

            if it % 10000 == 0:
                print("The Cost function for the iteration {}----->{}".format(it, cost))
                
    def predict(self, test_X):
        """
        :param test_X: feature values to predict.
        """
        # Insert constant ones for bias weights.
        test_X = np.insert(test_X, 0, 1, axis=1)

        y_pred = self.hypothesis(self.w, self.b, test_X)
        return y_pred

In [36]:
class l1_regularization:
    '''Regularization used for Lasso Regression'''
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        '''This will be retuned when we call this class.'''
        return self.lamda * np.sum(np.abs(weights))
    
    def derivation(self, weights):
        "Derivation of the regulariozation function."
        return self.lamda * np.sign(weights)
    
    
class l2_regularization:
    '''Regularization used for Ridge Regression'''
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        "This will be retuned when we call this class."
        return self.lamda * np.sum(np.square(weights))
    
    def derivation(self, weights):
        "Derivation of the regulariozation function."
        return self.lamda * 2 * (weights)

In [37]:
class LassoRegression(Regression):
    '''
    '''
    def __init__(self, lamda, learning_rate, iteration):
        '''
        Define the hyperparameters we are going to use in this model.
        :param lamda: Regularization factor.
        :param learning_rate: A samll value needed for gradient decent, default value id 0.1.
        :param iteration: Number of training iteration, default value is 10,000.
        '''
        self.regularization = l1_regularization(lamda)
        super(LassoRegression, self).__init__(learning_rate, iteration, self.regularization)

    def train(self, X, y):
        '''
        :param X: training data feature values ---> N Dimentional vector.
        :param y: training data target value -----> 1 Dimentional array.
        '''
        return super(LassoRegression, self).train(X, y)
    
    def predict(self, test_X):
        '''
        parma test_X: Value need to be predicted.
        '''
        return super(LassoRegression, self).predict(test_X)

In [38]:
# class RidgeRegression(Regression):
#     '''
#     Ridge Regression is one of the variance of the Linear Regression. This model doing the parameter learning 
#     and regularization at the same time. This model uses the l2-regularization. 
#     This is very similar to the Lasso regression.
#     * Regularization will be one of the soluions to the Overfitting.
#     * Overfitting happens when the model has "High Variance and low bias". So, regularization adds a little bias to the model.
#     * This model will try to keep the balance between learning the parameters and the complexity of the model( tries to keep the parameter having small value and small degree of palinamial).
#     * The Regularization parameter(lamda) controls how severe  the regularization is. 
#     * large lamda adds more bias , hence the Variance will go very small --> this may cause underfitting(Low bias and High Varinace).
#     * Lamda can be found by tial and error methos. 
#     '''
#     def __init__(self, lamda, learning_rate, iteration):
#         """
#         Define the hyperparameters we are going to use in this model.
#         :param lamda: Regularization factor.
#         :param learning_rate: A samll value needed for gradient decent, default value id 0.1.
#         :param iteration: Number of training iteration, default value is 10,000.
#         """
#         self.regularization = l2_regularization(lamda)
#         super(RidgeRegression, self).__init__(learning_rate, iteration, self.regularization)

#     def train(self, X, y):
#         """
#         :param X: training data feature values ---> N Dimentional vector.
#         :param y: training data target value -----> 1 Dimentional array.
#         """
#         return super(RidgeRegression, self).train(X, y)
#     def predict(self, test_X):
#         """
#         parma test_X: Value need to be predicted.
#         """
#         return super(RidgeRegression, self).predict(test_X)

In [39]:
def predict_scale_labels(y_scale_pred):
    '''
    Assign label 1 through 4
    
    arg:
    - y_scale_pred: vector of predicted y values
    
    Returns: vector of predicted y labels (1 through 4)
    '''
    # generate nx4 matrix where each column is 1,2,3,4
    m, _ = y_scale_pred.shape
    mat = np.tile(np.arange(4)+1, (m,1))
    
    # find lowest absolute distance
    return np.argmin(abs(mat - y_scale_pred), axis=1).reshape(m, 1)

### Estimate Lasso Model

In [40]:
param = {"lamda" : 0,
         "learning_rate" : 0.0000000000000000000001,
         "iteration" : 500}

lasso = LassoRegression(**param)
lasso.train(X, y_scale)

# predict
y_scale_pred = lasso.predict(X)

The Cost function for the iteration 10----->2959462394.2005224
The Cost function for the iteration 20----->2959462231.4092817
The Cost function for the iteration 30----->2959462068.6180515
The Cost function for the iteration 40----->2959461905.826829
The Cost function for the iteration 50----->2959461743.0356145
The Cost function for the iteration 60----->2959461580.24441
The Cost function for the iteration 70----->2959461417.4532146
The Cost function for the iteration 80----->2959461254.662028
The Cost function for the iteration 90----->2959461091.8708496
The Cost function for the iteration 100----->2959460929.079681
The Cost function for the iteration 110----->2959460766.288522
The Cost function for the iteration 120----->2959460603.4973702
The Cost function for the iteration 130----->2959460440.7062297
The Cost function for the iteration 140----->2959460277.9150968
The Cost function for the iteration 150----->2959460115.123973
The Cost function for the iteration 160----->2959459952.

In [ ]:
X_names = df.columns[(~df.columns.isin(["child_id", "mother_id", "treat_alike_scale", "treat_alike_binary"]))]

filt = (np.round(lasso.w[1:], 4)>0.0).flatten()
b_names = X_names[filt]
b_vals  = lasso.w[1:][filt]

print("---most important features---")
for n, v in zip(b_names, b_vals):
    print(n, ":", '%.08f' % v)

In [ ]:
print("---error rate---")
y_scale_pred_label = predict_scale_labels(y_scale_pred)
print(100*np.sum(y_scale != y_scale_pred_label)/len(y_scale))

print("---mean square error ---")
mse = (np.square(y_scale - y_scale_pred)).mean(axis=None)
mse

In [ ]:
# # visualize results
results = pd.DataFrame(data=np.hstack((y_scale, y_scale_pred)),
             columns=["y", "y_pred"])

sns.boxplot(data=results, x='y', y='y_pred').set(title='Prediction by Label')

# Lasso Package

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_sklearn = Lasso(alpha=0.0)
lasso_sklearn.fit(X, y_scale)

# predict the value
y_pred_sklearn = lasso_sklearn.predict(X)

In [ ]:
lasso.w

In [ ]:
lasso_sklearn.coef_

In [ ]:
np.max(y_pred_sklearn)